In [1]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

def load_images_labels(dataset_dir):
    images_dir = os.path.join(dataset_dir, "images")
    labels_dir = os.path.join(dataset_dir, "labels")
    
    data = {}
    image_data = {}
    
    for image_file in os.listdir(images_dir):
        if image_file.endswith(".jpg"):
            image_path = os.path.join(images_dir, image_file)
            label_file = image_file[:-3] + "txt"
            label_path = os.path.join(labels_dir, label_file)
            if os.path.exists(label_path):
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                if image is not None:
                    labels = []
                    with open(label_path, 'r') as file:
                        for line in file:
                            yolo_coords = line.strip().split(" ")
                            labels.append(yolo_coords)
                    data[image_file] = labels
                    image_data[image_file] = image
    return data, image_data





def extract_sign(image, yolo_coords):
    image_np = np.array(image)

    image_height, image_width = image_np.shape[:2]

    x_center, y_center, width, height = float(yolo_coords[1]) , float(yolo_coords[2]), float(yolo_coords[3]), float(yolo_coords[4])
    x_center *= image_width
    y_center *= image_height
    width *= image_width
    height *= image_height

    x_min = int(x_center - (width / 2))
    y_min = int(y_center - (height / 2))
    x_max = int(x_center + (width / 2))
    y_max = int(y_center + (height / 2))

    sign_roi = image_np[y_min:y_max, x_min:x_max]
    return sign_roi, int(yolo_coords[0])

if __name__ == "__main__":
    data, image_data = load_images_labels("archive/train")

    green_light_data = []
    red_ligh_data = []
    speed_limit_10 = []
    speed_limit_100 = []
    speed_limit_110 = []
    speed_limit_120 = []
    speed_limit_20 = []
    speed_limit_30 = []
    speed_limit_40 = []
    speed_limit_50 = []
    speed_limit_60 = []
    speed_limit_70 = []
    speed_limit_80 = []
    speed_limit_90 = []
    stop_sign_data = []

    sign_datas = {0 : green_light_data,1 : red_ligh_data,2: speed_limit_10,3: speed_limit_100, 4 : speed_limit_110,
                        5  : speed_limit_120,6 : speed_limit_20, 7 : speed_limit_30, 8 : speed_limit_40, 9: speed_limit_50
                        ,10 :  speed_limit_60, 11 : speed_limit_70,12 : speed_limit_80,13 : speed_limit_90, 14 : stop_sign_data}

    for i in range(len(data)):
            key = list(data.keys())[i]
            image = image_data[key]
            for j in range(len(data[key])):
                extracted_image, image_label = extract_sign(image, data[key][j])
                extracted_image = cv.resize(extracted_image, [64,64])
                extracted_image = cv2.cvtColor(extracted_image, cv2.COLOR_RGB2GRAY)
                sign_datas[image_label].append(extracted_image)

In [2]:
sign_datas_arrays = {}
for i in range(len(sign_datas.keys())):
    key = list(sign_datas.keys())[i]
    images_list = sign_datas[key]
    images_array = np.array(images_list)
    flattened_images = images_array.reshape(len(images_list), -1)
    sign_datas_arrays[i] = flattened_images


In [3]:
data = []
for i in range(15):
    for j in range(len((sign_datas_arrays[i]))):
        data.append(np.array(sign_datas_arrays[i][j]))
data = np.array(data)
data = data.T
final_data = []
for i in range(len(data)):
    temp = []
    for j in range(len(data[0])):
        temp.append(data[i][j])
    final_data.append(temp)
final_data = np.array(final_data)
final_data.shape

(4096, 4298)

In [4]:
U, S, V_t = np.linalg.svd(final_data.copy())

In [5]:
for i in range(len(S)):
    if S[i] < 2000:
        insig_vectors = i
        break

In [6]:
coordinates_of_vectors = []
inv = np.linalg.inv(U)
for i in range(0,len(final_data[0])):
    coordinates_of_vector = inv @ final_data[:, i:i+1]
    coordinates_of_vectors.append(coordinates_of_vector[:insig_vectors])

In [7]:
data_for_test, image_data = load_images_labels("archive/test")
green_light_test = []
red_ligh_test = []
speed_limit_10_test = []
speed_limit_100_test = []
speed_limit_110_test = []
speed_limit_120_test = []
speed_limit_20_test = []
speed_limit_30_test = []
speed_limit_40_test = []
speed_limit_50_test = []
speed_limit_60_test = []
speed_limit_70_test = []
speed_limit_80_test = []
speed_limit_90_test = []
stop_sign_data_test = []

sign_datas_test = {0 : green_light_test,1 : red_ligh_test,2: speed_limit_10_test,3: speed_limit_100_test, 4 : speed_limit_110_test,
                    5  : speed_limit_120_test,6 : speed_limit_20_test, 7 : speed_limit_30_test, 8 : speed_limit_40_test, 9: speed_limit_50_test
                    ,10 :  speed_limit_60_test, 11 : speed_limit_70_test,12 : speed_limit_80_test,13 : speed_limit_90_test, 14 : stop_sign_data_test}

for i in range(len(data_for_test)):
        key = list(data_for_test.keys())[i]
        image = image_data[key]
        for j in range(len(data_for_test[key])):
            extracted_image, image_label = extract_sign(image, data_for_test[key][j])
            extracted_image = cv.resize(extracted_image, [64,64])
            extracted_image = cv2.cvtColor(extracted_image, cv2.COLOR_RGB2GRAY)
            sign_datas_test[image_label].append(extracted_image)

In [8]:
sign_datas_arrays_test = {}    
for i in range(len(sign_datas_test.keys())):
    key = list(sign_datas_test.keys())[i]
    images_list = sign_datas_test[key]
    images_array = np.array(images_list)
    flattened_images = images_array.reshape(len(images_list), -1)
    sign_datas_arrays_test[i] = flattened_images


In [9]:
data_test = []
for i in range(15):
    for j in range(len((sign_datas_arrays_test[i]))):
        data_test.append(np.array(sign_datas_arrays_test[i][j]))
data_test = np.array(data_test)
data_test = data_test.T
final_data_test = []
for i in range(len(data_test)):
    temp = []
    for j in range(len(data_test[0])):
        temp.append(data_test[i][j])
    final_data_test.append(temp)
final_data_test = np.array(final_data_test)
final_data_test.shape


(4096, 770)

In [10]:
lst_of_labels_train = [542,585, 19, 267,101,252,285,334,235,283,301,318,323,168,285]
labels_train = []
for i,k in enumerate(lst_of_labels_train):
    for j in range(k):
        labels_train.append(i)
labels_train[1501]
lst_of_labels_test = [110, 94, 3, 46, 21, 44, 46, 60,53, 50, 45, 53, 61, 34, 50]
labels_test = []
for i,k in enumerate(lst_of_labels_test):
    for j in range(k):
        labels_test.append(i)


In [11]:
import heapq
true_guessed = 0
guesses = []
predicitons = []

for i in range(len(final_data_test[0])):
    image = final_data_test[:, i:i+1]
    coords_of_image = inv @ image
    lengths =  []
    list_of_tuples = []
    for cords in coordinates_of_vectors:
        vector_cords = coords_of_image[:insig_vectors] - cords[:insig_vectors]
        lengths.append(np.linalg.norm(vector_cords, ord=2))
    for j in range(len(lengths)):
        list_of_tuples.append(tuple([lengths[j], labels_train[j]]))
    smallest_10_tuples = heapq.nsmallest(1, list_of_tuples, key=lambda x: x[0])
    checker = [0] * 15
    for j in range(1):
        tup = smallest_10_tuples.pop()
        checker[tup[1]] += 1
    pred = checker.index(max(checker))
    predicitons.append(pred)
    if pred == labels_test[i]:
        true_guessed += 1
        guesses.append(tuple([pred, True, labels_test[i], image]))
    else:
        guesses.append(tuple([pred, False, labels_test[i], image]))


In [12]:
true_guessed

576

In [13]:
print("Accuracy = " + str(true_guessed / len(final_data_test[0])))

Accuracy = 0.7480519480519481
